<a href="https://colab.research.google.com/github/Sim98B/MushroomVision/blob/main/MushD4/MushDetector4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mushroom Detector

## Setup

In [1]:
# Importing main libraries
try:
  import torch
  import torchvision
  import torchinfo
except:
  print("[INFO] Torch and Torchvision not installed, downloading")
  !pip install -q torch torchvision torchinfo
import torch
print(f"torch: {torch.__version__}")
import torchvision
print(f"torchvision: {torchvision.__version__}")
import torchinfo
print(f"torchinfo: {torchinfo.__version__}")

# Utilities
import os
from pathlib import Path
import requests
import shutil
from timeit import default_timer as timer
import itertools
import warnings
warnings.filterwarnings("ignore")

# Manipulation
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Neural networks
from torch import nn
from torchvision import transforms
import torchvision.models
from torchinfo import summary

# Metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Cloning GitHub repository
!git clone https://github.com/Sim98B/MushroomVision
!mv MushroomVision/Modules/Libraries .

# Custom functions
from Libraries import data, extractors, model_ops, utils

shutil.rmtree("MushroomVision")

[INFO] Torch and Torchvision not installed, downloading
torch: 2.1.0+cu118
torchvision: 0.16.0+cu118
torchinfo: 1.8.0
Cloning into 'MushroomVision'...
remote: Enumerating objects: 64136, done.
remote: Counting objects: 100% (64136/64136), done.
remote: Compressing objects: 100% (63021/63021), done.
remote: Total 64136 (delta 1041), reused 64092 (delta 1012), pack-reused 0
Receiving objects: 100% (64136/64136), 700.75 MiB | 43.48 MiB/s, done.
Resolving deltas: 100% (1041/1041), done.
Updating files: 100% (1739/1739), done.


In [2]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Data: [CIFAR100](https://www.cs.toronto.edu/~kriz/cifar.html)
The best way to identify those photos that contain photos is to use photos of 100 different subjects.

In [3]:
tensor_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform = tensor_transform)
test_data = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform = tensor_transform)

print(f"{len(train_data)} images for train divided into {len(train_data.classes)} classes")
print(f"{len(test_data)} images for train divided into {len(test_data.classes)} classes")

100%|██████████| 169001437/169001437 [00:01<00:00, 100663904.15it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
50000 images for train divided into 100 classes
10000 images for train divided into 100 classes


In [4]:
# We have a mushroom class in this dataset, at index 51
for i in range(len(train_data.classes)):
  if train_data.classes[i] == "mushroom":
    print(f"{i+1}. {train_data.classes[i].upper()}")

class_names = train_data.classes

52. MUSHROOM


## Baseline
As in the [PyTorch example](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) we are going to use a simple convolutional baseline model.

In [5]:
# Creating a simple convolutional model

utils.set_seed()

class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 100)

  def forward(self, x):
    x = self.pool(torch.nn.functional.relu(self.conv1(x)))
    x = self.pool(torch.nn.functional.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = torch.nn.functional.relu(self.fc1(x))
    x = torch.nn.functional.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = Classifier()
model

Classifier(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=100, bias=True)
)

In [6]:
# Dataloaders
train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                           batch_size = 32,
                                           shuffle = True,
                                           num_workers = os.cpu_count(),
                                           pin_memory = True)

test_loader = torch.utils.data.DataLoader(dataset = test_data,
                                          batch_size = 32,
                                          shuffle = False,
                                          num_workers = os.cpu_count(),
                                          pin_memory = True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [7]:
# Trainig the model for 30 epochs
results = model_ops.train(model = model,
                          train_dataloader = train_loader,
                          test_dataloader = test_loader,
                          loss_function = criterion,
                          optimizer = optimizer,
                          metric = "accuracy",
                          epochs = 30,
                          device = device,
                          verbose = 1)

Training:   0%|          | 0/30 [00:00<?, ?it/s]

## Some help from [Kaggle](https://www.kaggle.com/code/yiweiwangau/cifar-100-resnet-pytorch-75-17-accuracy)